<a href="https://colab.research.google.com/github/itsZENR/Binance/blob/main/BinanceRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
#@title Библиотеки
import pandas as pd
import numpy as np
import random
import requests
import csv
import datetime
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime

In [3]:
BINANCE_CSV_1h_ARBUSDT = "/content/binance_quotes_1h_ARBUSDT.csv"
# BINANCE_CSV_1h_BELUSDT = "/content/binance_quotes_1h_BELUSDT.csv"
# BINANCE_CSV_1h_BTCUSDT = "/content/binance_quotes_1h_BTCUSDT.csv"
# BINANCE_CSV_1h_ETHUSDT = "/content/binance_quotes_1h_ETHUSDT.csv"
# BINANCE_CSV_1h_XRPUSDT = "/content/binance_quotes_1h_XRPUSDT.csv"

In [4]:
# Загружаем CSV-файл с данными и преобразуем данные в DataFrame
df1 = pd.read_csv(BINANCE_CSV_1h_ARBUSDT)[['open', 'close']]
# df2 = pd.read_csv(BINANCE_CSV_1h_BELUSDT)[['open', 'close']]
# df3 = pd.read_csv(BINANCE_CSV_1h_BTCUSDT)[['open', 'close']]
# df4 = pd.read_csv(BINANCE_CSV_1h_ETHUSDT)[['open', 'close']]
# df5 = pd.read_csv(BINANCE_CSV_1h_XRPUSDT)[['open', 'close']]
df1

,open,close
0,0.5000,1.3250
1,1.3251,1.3040
2,1.3040,1.2890
3,1.2890,1.3273
4,1.3274,1.4300
...,...,...
542,1.6401,1.6591
543,1.6590,1.6450
544,1.6450,1.6723
545,1.6725,1.6621


In [54]:
#@title Среда
class Binance:
    def __init__(self, df):
        self.balance = 100
        self.state = df
        # Открытые позиции
        self.open_positions = None
        # Предсказанное направление цены
        self.price_movement = ""

        self.quantityBet = 0
        self.quantity = 0

        self.priceBuy = 0
        self.priceBuyStop = 0
        self.priceSell = 0
        self.priceSellStop = 0

        # self.info_positions = []
        
        # Торгуем % от баланса
        self.balanceProcent = 100
        self.balanceProcent = self.balanceProcent / 100


    def quantity_lot(self, balance, price):
      quantity = 0
      print('balance, price', balance, price)
      if self.balance > 0:
        # торгуем % баланса
        self.balance = self.balance * self.balanceProcent
        # расчет размера позиции на покупку/продажу
        self.quantity = self.balance / price
        print('self.quantity', self.quantity)
        self.quantity = math.floor(self.quantity * 10) / 10.0
        # self.quantity = np.around(self.quantity, 2)
        print('self.quantity2222', self.quantity)

      return self.quantity

    def buy(self, price):
        self.quantity = self.quantity_lot(self.balance, price)
        self.balance -= self.quantity * price
        # Запоминаем цену покупки/продажи
        self.priceBuy = price
        #  получаем id покупки
        # id = 0
        # Указываем (объем, цена одной, айди покупки)
        # self.info_positions.append([self.quantity, self.priceBuy, id])

        self.priceBuyStop = price
        # Запоминаем количество лота
        self.quantityBet = self.quantity
        # обновляем значение переменной для открытых позиций
        self.open_positions = "rise"

    def sell(self, price):
        self.quantity = self.quantity_lot(self.balance, price)
        print('self.balance1', self.balance, price, self.quantity)
        self.balance -= self.quantity * price
        print('self.balance1', self.balance, price, self.quantity)

        
        # Запоминаем цену покупки/продажи
        self.priceSell = price
        #  получаем id покупки
        # id = 0

        # Указываем (объем, цена одной, айди покупки)
        # self.info_positions.append([self.quantity, self.priceSell, id])
        self.priceSellStop = price
        # Запоминаем количество лота
        self.quantityBet = self.quantity
        # обновляем значение переменной для открытых позиций
        self.open_positions = "fall"


    def step(self, action, j):
        reward = 0  

        # Берем последнюю цену
        price = self.state['close'].iloc[j]

        # проверяем условия для открытия или закрытия позиций
        if action > 0 and self.open_positions is None: # если предсказан рост цены и нет открытых позиций, то открываем позицию на покупку
            self.buy(price)


        elif  action < 0 and self.open_positions is None: # если предсказано падение цены и нет открытых позиций, то открываем позицию на продажу
            self.sell(price)
            

        elif action > 0 and self.open_positions == "fall": # если предсказан рост цены и есть открытая позиция на продажу, то закрываем ее
            # print(f'Закрываем ордер ')
            # ======== (Стоимость захода в позицию) + (Стоимость разницы)
            self.balance += self.quantityBet * self.priceSell + self.quantityBet * (self.priceSell - price)
            reward = self.quantityBet * (self.priceSell - price)
            # self.balance += self.quantityBet * (2 * self.priceSell - price)
            self.buy(price)


        elif action < 0 and self.open_positions == "rise": # если предсказано падение цены и есть открытая позиция на покупку, то закрываем ее
            # print(f'Закрываем ордер')
            self.balance += self.quantityBet * price
            reward = self.quantityBet * -(self.priceBuy - price)
            self.sell(price) 
        

        return self.state.iloc[j+1], reward

In [ ]:
ARB = Binance(df1)
ARB.state.iloc[1]

open     1.3251
close    1.3040
Name: 1, dtype: float64

In [6]:
# Определение модели Q-функции
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='tanh', input_shape=(2,)),
    tf.keras.layers.Dense(3)
])

model.compile(optimizer='Adam', loss=tf.keras.losses.BinaryCrossentropy())

In [55]:
# Определение функции выбора действия
def choose_action(state, epsilon):
    if np.random.random() < epsilon:
        # Случайное действие
        # action = np.random.choice(np.where(env.state.flatten() == 0)[0])
        action = np.random.choice([-1,0,1])
    else:
        # Выбор действия с наибольшим значением Q-функции
        q_values = model.predict(state.reshape(1, -1))
        action = np.argmax(q_values[0] * (env.state.flatten() == 0))
        print('q_values', q_values)
    return action

# Определение функции обновления Q-функции
def update_q_function(state, action, reward, next_state, alpha, gamma):
    # q_values = model.predict(state.reshape(1, -1))
    q_values = model.predict(np.array(state).reshape(1, -1))
    next_q_values = model.predict(np.array(next_state).reshape(1, -1))
    td_target = reward + gamma * np.amax(next_q_values)
    td_error = td_target - q_values[0][action]
    q_values[0][action] += alpha * td_error
    model.fit(np.array(state).reshape(1, -1), q_values)

# Основной цикл обучения
num_episodes = 100
epsilon = 1.0
epsilon_decay = 0.9995
alpha = 0.01
gamma = 0.99

for i in range(num_episodes):
    print('-'*100)
    env = Binance(df1)
    for j in range(len(env.state)):
        action = choose_action(env.state.iloc[j], epsilon)
        print("action", action)
        print('balance', env.balance)
        next_state, reward = env.step(action, j)
        print('reward', reward)

        update_q_function(env.state.iloc[j], action, reward, next_state, alpha, gamma)
        state = next_state

    epsilon *= epsilon_decay

----------------------------------------------------------------------------------------------------
action -1
balance 100
balance, price 100 1.325
self.quantity 75.47169811320755
self.quantity2222 75.4
self.balance1 100.0 1.325 75.4
self.balance1 0.09499999999999886 1.325 75.4
reward 0
1/1 [==============================] - 0s 7ms/step - loss: -1.1663
action 1
balance 0.09499999999999886
balance, price 101.5834 1.304
self.quantity 77.90138036809816
self.quantity2222 77.9
reward 1.5833999999999933
1/1 [==============================] - 0s 7ms/step - loss: -1.3810
action 0
balance 0.0017999999999886995
reward 0
1/1 [==============================] - 0s 9ms/step - loss: -1.3542
action 1
balance 0.0017999999999886995
reward 0
1/1 [==============================] - 0s 10ms/step - loss: -1.3728
action -1
balance 0.0017999999999886995
balance, price 111.3988 1.43
self.quantity 77.90125874125874
self.quantity2222 77.9
self.balance1 111.3988 1.43 77.9
self.balance1 0.0017999999999886995 1.43 7

KeyboardInterrupt: ignored